# `Hit the Gym: MiniProject for RL Games CPP`

The exercises in this notebook are meant for CVI aspirants who wish to work on the RL Games CPP. 

**About the project:** The goal of the RL Games project is to explore and analyse current Reinforcement Learning Methods by deploying them on Atari games, Minesweeper, slither.io etc. Further, the techniques developed can be used to attack more complex problems such as Reconnaissance Blind Chess and Autonomous Driving.

In this notebook, you'll use a popular Reinforcement Learning Technique called Q-learning to train an agent to play simple games using the python library OpenAI Gym. 

You may refer to the following while coding:

Python reference: https://bit.ly/3ajUalZ and https://bit.ly/2UgiZKa

OpenAI Gym Documentation: https://gym.openai.com/docs/


### `RL Basics:`
Reinforcement learning is an approach used in Machine Learning where the agent is allowed to interact with the system to learn its behaviour and come up with an optimal startegy to achieve an objective. The agent models the problem as a probabilistic state machine (a graph where the transition from one node to another has a probability distribution). Nodes in the model graph are called states and a transition from one state to another is called an action. Each state transition (which is a (state, action) pair) has a corresponding reward or penalty. The goal of the RL agent is to maximise the reward. 

Training an RL agent from scratch requires us to model the state space and the action space. In addition, we must also come up with suitable rewards for each state transition. The RL agent estimates this reward structure and executes actions so as to maximise them. The final performance of the RL agent is heavily dependent on how the system is modelled. Luckily for us, we **do not need to get into the mathematics** of Reinforcement Learning right now, thanks to the Python library Gym.


Gym offers many in-built RL environments which you can use to play around with. These environments are Python classes with their state spaces, action spaces and rewards pre-defined. You will use two such environments (Taxi-v3 and FrozenLake8x8-v0) to train an agent accomplish a goal. You can find the documentation for these environments here:

Taxi-v3 Documentation: https://gym.openai.com/envs/Taxi-v3/  

FrozenLake Documentation: https://gym.openai.com/envs/FrozenLake8x8-v0/  

To create a gym environment of 'Taxi-v3' you do this:

In [2]:
import gym
import numpy as np  
# Create an environment of Taxi-v3:
env = gym.make('Taxi-v3').env 
env.render()
print(env.s)
print(env.observation_space)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

41
Discrete(500)


env.s is the current state of the environment.

env.observation_space() returns the type and size of the observation or state space. Discrete implies that the states are discrete and not continuous. For games like Pacman, Gym uses another type of state space- Box- due to the large number of states. 

**Solving the Problem:** Now the goal of this game is to train the taxi to efficiently pick the passenger from the blue spot and drop them at the purple spot. The taxi can do the following actions: Move Up, Move Down, Move to the Left, Move to the Right, Pickup, Dropoff. The reward structure is such:
1. -10 points for illegal dropoff/pickup actions
2. +20 points if the passenger is dropped off at the correct location
3. -1 point for every other action

A paleolithic approach to this problem would be to pick an action at random and execute it. Eventually the passenger would get picked up and then dropped off at the correct location.

In [3]:
state = env.reset()
epochs = 0
penalty, reward = 0, 0  # Penalty records the number of times the agent hits a wall
frames = []
done = False# Find out the role of 'done' and complete the statement for its initial condition 
while not done:
    action = env.action_space.sample()
    new_state, reward, done, info = env.step(action)
    if new_state == state:
      penalty+=1
    state = new_state

    frames.append({'frame': env.render(mode='ansi'), 'state': state, 'action': action, 'reward': reward})
    epochs += 1
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalty))

Timesteps taken: 1643
Penalties incurred: 896


Run the following cell to visualise the performance of the agent. It won't come as a surprise to see that the approach is quite bad. It is so because, the agent has no memory of the past and hence learns nothing. 

In [4]:
from IPython.display import clear_output
from time import sleep
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait = True)
        print(frame['frame'])
        print(f"Timestep: {i+1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        #sleep(0.1)
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1643
State: 0
Action: 5
Reward: 20


## `Enter Q-learning`

A popular technique used in Reinforcement Learning is to have the agent maintain an estimate of the rewards that it would gain from executing a particular state transition (called the Q-table) which is updated after each step based on the reward obtained. The agent picks the action that yields the maximum reward at that particular state. That is, if a particular state transition ((state, action) pair) resulted in a good reward, the agent is better off repeating that action whenever that state is attained. 

**TASK 1:**

Find out the update rule for Q-learning and implement Q-learning for the Taxi-v3 problem. Also, find out what 'exploration versus exploitation' is and use a suitable way to optimise on exploration and exploitation.

You would need to set a few hyperparameters- learning rate ($\alpha$), reward decay rate ($\gamma$), number of episodes and exploration probability($\epsilon$). Obtain the performance characteristics of the agent (that is, number of epochs per episode and average number of penalties per episode) for ($\alpha$, $\gamma$, episodes, $\epsilon$) = (0.6, 0.9, 1000, 0.4)

In [6]:
import numpy as np
import random
q_table = np.zeros([env.observation_space.n, env.action_space.n])
(alpha, gamma, episodes, epsilon) = (0.6, 0.9, 1000, 0.4)
total_epochs =0
total_penalty = 0
illegal_episode =[]
for i in range(episodes):
  state = env.reset()
  epochs = 0
  penalty, reward_tot = 0, 0  # Penalty records the number of times the agent hits a wall
  illegal =0
  
  done = False# Find out the role of 'done' and complete the statement for its initial condition 
  while not done:
    rand = random.uniform(0,1)
    if rand < epsilon:
      action = env.action_space.sample()
    if rand >= epsilon:
      action = np.argmax(q_table[state])
    new_state, reward, done, info = env.step(action)
    if new_state == state:
      penalty+=1

    if reward == -10:
      illegal+=1

    epochs += 1

    oldq = q_table[state, action]
    new_state_max = np.max(q_table[new_state])
          
    newq = (1 - alpha) * oldq + alpha * (reward + gamma * new_state_max)
    q_table[state, action] = newq
    state = new_state
    epochs+=1
  illegal_episode.append(illegal)
  total_penalty+=penalty
  total_epochs+=epochs

total_penalty/=episodes
total_epochs/=episodes


**TASK 2:**

Now that you have trained the agent on Taxi-v3, try Q-learning on the FrozenLake8x8-v0 environment. After training, obtain the following performance characteristics- number of epochs per episode and average number of wins. What can you do to improve the performance of the agent?

In [28]:

env = gym.make('FrozenLake8x8-v0').env 
q_table = np.zeros([env.observation_space.n, env.action_space.n])
(alpha, gamma, episodes, epsilon) = (0.72, 0.5, 400, 1)
total_epochs =0
tot_wins =0
max_epochs = 100
check =[]
for i in range(episodes):
  state = env.reset()
  epochs = 0
  
  done = False 
  while not done:
    #wins = False
    rand = random.uniform(0,1)
    if rand < epsilon:
      action = env.action_space.sample()
    if rand >= epsilon:
      action = np.argmax(q_table[state])
    new_state, reward, done,__ = env.step(action)

    if reward ==1: # when it reaches the goal
      #wins=True
      tot_wins+=1
    elif reward==0 and done: # when it falls into a hole
      reward=-10
    else: # for any other movement
      reward=-1
    
    

    oldq = q_table[state, action]
    new_state_max = np.max(q_table[new_state])
          
    newq = (1 - alpha) * oldq + alpha * (reward + gamma * new_state_max)
    q_table[state, action] = newq
    state = new_state
    epochs+=1
    

  total_epochs+=epochs
  epsilon*=0.3

'''
To improve performance let the loop run for more episodes and make the epsilon a decaying value so initially the model mostly explores and
later the model mostly exploits. Also here I have changed the reward system so that the agent does not get stuck just moving back and forth and actually moves towards
the goal which was not motivated enough in the previous system of rewards. This also improves the performance.
From trial and error, the decay value of epsilon is kept to be very small for improved performance. A possible explanation is a lot of possible 
exploration paths are there but they dont lead to a goal any different. So it makes sense that if the agent finds a path that safely leads to the goal
it should stick to it and not explore too much. Still some exploration could be favoured to find shorter paths. If no of episodes is to be increased, this
value should also be changed.
'''

'\nTo improve performance let the loop run for more episodes and make the epsilon a decaying value so initially the model mostly explores and\nlater the model mostly exploits. Also here I have changed the reward system so that the agent does not get stuck just moving back and forth and actually moves towards\nthe goal which was not motivated enough in the previous system of rewards. This also improves the performance.\nFrom trial and error, the decay value of epsilon is kept to be very small for improved performance. A possible explanation is a lot of possible \nexploration paths are there but they dont lead to a goal any different. So it makes sense that if the agent finds a path that safely leads to the goal\nit should stick to it and not explore too much. Still some exploration could be favoured to find shorter paths. If no of episodes is to be increased, this\nvalue should also be changed.\n'

In [29]:
print(f'won {tot_wins} out of {episodes} episodes') #0.3 decay

won 308 out of 400 episodes


## `Further Motivation`

In case you are curious about the mathematics of Reinforcement Learning, you check the following resources out:

RL Lectures by Dr. David Silver: http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching.html

Medium Blog Post on RL techniques: https://medium.com/@jonathan_hui/rl-introduction-to-deep-reinforcement-learning-35c25e04c199

Deep Neural Networks (useful for Deep RL): http://cs231n.github.io/

If you are facing any issue or difficulty, you may PM (on WhatsApp):

Arjun: 97392 19819